# T1547.006 - Kernel Modules and Extensions
Adversaries may modify the kernel to automatically execute programs on system boot. Loadable Kernel Modules (LKMs) are pieces of code that can be loaded and unloaded into the kernel upon demand. They extend the functionality of the kernel without the need to reboot the system. For example, one type of module is the device driver, which allows the kernel to access hardware connected to the system.(Citation: Linux Kernel Programming) 

When used maliciously, LKMs can be a type of kernel-mode [Rootkit](https://attack.mitre.org/techniques/T1014) that run with the highest operating system privilege (Ring 0).(Citation: Linux Kernel Module Programming Guide) Common features of LKM based rootkits include: hiding itself, selective hiding of files, processes and network activity, as well as log tampering, providing authenticated backdoors, and enabling root access to non-privileged users.(Citation: iDefense Rootkit Overview)

Kernel extensions, also called kext, are used in macOS to load functionality onto a system similar to LKMs for Linux. Since the kernel is responsible for enforcing security and the kernel extensions run as apart of the kernel, kexts are not governed by macOS security policies. Kexts are loaded and unloaded through <code>kextload</code> and <code>kextunload</code> commands. Kexts need to be signed with a developer ID that is granted privileges by Apple allowing it to sign Kernel extensions. Developers without these privileges may still sign kexts but they will not load unless SIP is disabled. If SIP is enabled, the kext signature is verified before being added to the AuxKC.(Citation: System and kernel extensions in macOS)

Since macOS Catalina 10.15, kernel extensions have been deprecated in favor of System Extensions. However, kexts are still allowed as "Legacy System Extensions" since there is no System Extension for Kernel Programming Interfaces.(Citation: Apple Kernel Extension Deprecation)

Adversaries can use LKMs and kexts to conduct [Persistence](https://attack.mitre.org/tactics/TA0003) and/or [Privilege Escalation](https://attack.mitre.org/tactics/TA0004) on a system. Examples have been found in the wild, and there are some relevant open source projects as well.(Citation: Volatility Phalanx2)(Citation: CrowdStrike Linux Rootkit)(Citation: GitHub Reptile)(Citation: GitHub Diamorphine)(Citation: RSAC 2015 San Francisco Patrick Wardle)(Citation: Synack Secure Kernel Extension Broken)(Citation: Securelist Ventir)(Citation: Trend Micro Skidmap)

## Atomic Tests

### Atomic Test #1 - Linux - Load Kernel Module via insmod
This test uses the insmod command to load a kernel module for Linux.

**Supported Platforms:** linux

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `bash`!
##### Description: The kernel module must exist on disk at specified location

##### Check Prereq Commands:
```bash
if [ -f /tmp/T1547.006/T1547006.ko ]; then exit 0; else exit 1; fi;

```
##### Get Prereq Commands:
```bash
if [ ! -d /tmp/T1547.006 ]; then mkdir /tmp/T1547.006; touch /tmp/T1547.006/safe_to_delete; fi;
cp PathToAtomicsFolder/T1547.006/src/* /tmp/T1547.006/
cd /tmp/T1547.006; make
if [ ! -f /tmp/T1547.006/T1547006.ko ]; then mv /tmp/T1547.006/T1547006.ko /tmp/T1547.006/T1547006.ko; fi;

```

In [ ]:
Invoke-AtomicTest T1547.006 -TestNumbers 1 -GetPreReqs

#### Attack Commands: Run with `bash`
```bash
sudo insmod /tmp/T1547.006/T1547006.ko
```

In [ ]:
Invoke-AtomicTest T1547.006 -TestNumbers 1

#### Cleanup: 
```bash
sudo rmmod T1547006
[ -f /tmp/T1547.006/safe_to_delete ] && rm -rf /tmp/T1547.006
```

In [ ]:
Invoke-AtomicTest T1547.006 -TestNumbers 1 -Cleanup

### Atomic Test #2 - MacOS - Load Kernel Module via kextload and kmutil
This test uses the kextload and kmutil commands to load and unload a MacOS kernel module.

**Supported Platforms:** macos

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `bash`!
##### Description: The kernel module must exist on disk at specified location

##### Check Prereq Commands:
```bash
if [ -d /Library/Extensions/SoftRAID.kext ] ; then exit 0; else exit 1 ; fi

```
##### Get Prereq Commands:
```bash
exit 1

```

In [ ]:
Invoke-AtomicTest T1547.006 -TestNumbers 2 -GetPreReqs

#### Attack Commands: Run with `bash`
```bash
set -x
sudo kextload /Library/Extensions/SoftRAID.kext
kextstat 2>/dev/null | grep SoftRAID
sudo kextunload /Library/Extensions/SoftRAID.kext
sudo kmutil load -p /Library/Extensions/SoftRAID.kext
kextstat 2>/dev/null | grep SoftRAID
sudo kmutil unload -p /Library/Extensions/SoftRAID.kext
```

In [ ]:
Invoke-AtomicTest T1547.006 -TestNumbers 2

### Atomic Test #3 - MacOS - Load Kernel Module via KextManagerLoadKextWithURL()
This test uses the IOKit API to load a kernel module for macOS.
Harcoded to use SoftRAID kext

**Supported Platforms:** macos

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `bash`!
##### Description: The kernel module must exist on disk at specified location

##### Check Prereq Commands:
```bash
if [ -f "/tmp/T1547006_iokit_loader" ]; then exit 0 ; else exit 1; fi

```
##### Get Prereq Commands:
```bash
cc -o /tmp/T1547006_iokit_loader PathToAtomicsFolder/T1547.006/src/macos_kextload.c -framework IOKit -framework Foundation

```

In [ ]:
Invoke-AtomicTest T1547.006 -TestNumbers 3 -GetPreReqs

#### Attack Commands: Run with `bash`
```bash
sudo /tmp/T1547006_iokit_loader
kextstat 2>/dev/null | grep SoftRAID
sudo kextunload /Library/Extensions/SoftRAID.kext
```

In [ ]:
Invoke-AtomicTest T1547.006 -TestNumbers 3

#### Cleanup: 
```bash
rm -f /tmp/T1547006_iokit_loader
```

In [ ]:
Invoke-AtomicTest T1547.006 -TestNumbers 3 -Cleanup

### Atomic Test #4 - Snake Malware Kernel Driver ComadminThe following Atomic Test will write an file, comadmin.dat, to disk. From the report, Snake's installer drops the kernel driver and a custom DLL which is used to load the driver into a
single AES encrypted file on disk. Typically, this file is named “comadmin.dat” and is stored in the %windows%\system32\Com directory. 
This Atomic Test will write a hardcoded named file to disk in the com directory named comadmin.dat.
[Snake Malware - CISA](https://media.defense.gov/2023/May/09/2003218554/-1/-1/0/JOINT_CSA_HUNTING_RU_INTEL_SNAKE_MALWARE_20230509.PDF)  
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `powershell`
```powershell
$examplePath = Join-Path $env:windir "system32\Com"; if (-not (Test-Path $examplePath)) { New-Item -ItemType Directory -Path $examplePath | Out-Null }; $exampleName = "comadmin.dat"; $exampleFullPath = Join-Path $examplePath $exampleName; $randomBytes = New-Object Byte[] 0x1000; (New-Object Random).NextBytes($randomBytes); [System.IO.File]::WriteAllBytes($exampleFullPath, $randomBytes)
```

In [ ]:
Invoke-AtomicTest T1547.006 -TestNumbers 4

#### Cleanup: 
```powershell
$examplePath = Join-Path $env:windir "system32\Com"; $exampleName = "comadmin.dat"; $exampleFullPath = Join-Path $examplePath $exampleName; if (Test-Path $exampleFullPath) { Remove-Item $exampleFullPath -Force }
```

In [ ]:
Invoke-AtomicTest T1547.006 -TestNumbers 4 -Cleanup

## Detection
Loading, unloading, and manipulating modules on Linux systems can be detected by monitoring for the following commands: <code>modprobe</code>, <code>insmod</code>, <code>lsmod</code>, <code>rmmod</code>, or <code>modinfo</code> (Citation: Linux Loadable Kernel Module Insert and Remove LKMs) LKMs are typically loaded into <code>/lib/modules</code> and have had the extension .ko ("kernel object") since version 2.6 of the Linux kernel. (Citation: Wikipedia Loadable Kernel Module)

Adversaries may run commands on the target system before loading a malicious module in order to ensure that it is properly compiled. (Citation: iDefense Rootkit Overview) Adversaries may also execute commands to identify the exact version of the running Linux kernel and/or download multiple versions of the same .ko (kernel object) files to use the one appropriate for the running system.(Citation: Trend Micro Skidmap) Many LKMs require Linux headers (specific to the target kernel) in order to compile properly. These are typically obtained through the operating systems package manager and installed like a normal package. On Ubuntu and Debian based systems this can be accomplished by running: <code>apt-get install linux-headers-$(uname -r)</code> On RHEL and CentOS based systems this can be accomplished by running: <code>yum install kernel-devel-$(uname -r)</code>

On macOS, monitor for execution of <code>kextload</code> commands and user installed kernel extensions performing abnormal and/or potentially malicious activity (such as creating network connections). Monitor for new rows added in the <code>kext_policy</code> table. KextPolicy stores a list of user approved (non Apple) kernel extensions and a partial history of loaded kernel modules in a SQLite database, <code>/var/db/SystemPolicyConfiguration/KextPolicy</code>.(Citation: User Approved Kernel Extension Pike’s)(Citation: Purves Kextpocalypse 2)(Citation: Apple Developer Configuration Profile)
